# 간단한 모델 생성
- XGB
- LGBM
- 'X,Y + 버스 + 지하철' 전처리된 데이터 사용
- 전처리 안 된 feature는 일단 삭제

In [2]:
!pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/f2/3d/4f152cf694aec100ab63b4a5547f2dbfbea59ab39d9375c89bed9775e47d/lightgbm-4.4.0-py3-none-manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.7 MB/s eta 0:00:0000:0100:01


In [90]:
!pip install joblib

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import gc

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

import joblib

# 맥에서 글씨 깨짐 방지
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# Data Load

In [57]:
df = pd.read_csv('../data/28_target_outlier.csv')

/tmp/ipykernel_472052/1477215302.py:1: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/28_target_outlier.csv')


In [58]:
df.shape

(1128094, 103)

In [59]:
# 일부 feature 제거 - 기준은 내마음대로

df.drop(['해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호',
         'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
         'k-사용검사일-사용승인일', 'k-관리비부과면적', 'k-전용면적별세대현황(60이하)', 'k-전용면적별세대현황(60~85이하)',
         'k-85~135이하', 'k-135초과', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태',
         '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일'], axis=1, inplace=True)

In [60]:
df.shape

(1128094, 74)

In [61]:
df.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도',
       '도로명', 'k-관리방식', 'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적', '세대전기계약방법',
       '청소비관리형태', '건축면적', '주차대수', '좌표X', '좌표Y', 'target', 'is_test', '시군구 번지',
       '좌표X,좌표Y', '가장 가까운 거리', 'index', '역사_ID', '역사명', '호선', '위도', '경도',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '가장 가까운 버스 정류장 index',
       '가장 가까운 버스 정류장 노드 ID', '가장 가까운 버스 정류소번호', '가장 가까운 버스 정류소명',
       '가장 가까운 버스 정류소 타입', '가장 가까운 버스 정류장 X좌표', '가장 가까운 버스 정류장 Y좌표',
       '인근 버스 정류장 개수', '계약년', '계약월', 'GDP', '한국은행 기준금리', '기대 인플레이션', '지가지수',
       '아파트 인허가', '미분양', '거래량', '건설사 랭킹', '구', '동', '구별 지가지수', '공시지가 평균',
       '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수', '매매가격 지수 증감률',
       '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리', '건설사 카테고리',
       '가장 가까운 다리와의 거리', '가장 가까운 다리 index', '가장 가까운 다리', '인근 한강다리 개수',
       '인근 다리 개수 1개이상 3개 이하', '학군'],
      dtype='object')

In [62]:
# 추가로 제거가 필요한 feature 제거
df.drop(['시군구', '번지', '본번', '부번', '아파트명', '도로명', 'k-관리방식', 'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적',
         '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '시군구 번지', '좌표X,좌표Y', 'index', '역사_ID', '역사명', '위도', '경도',
         '가장 가까운 버스 정류장 index', '가장 가까운 버스 정류장 노드 ID', '가장 가까운 버스 정류소번호', '가장 가까운 버스 정류소명',
         '가장 가까운 버스 정류소 타입', '가장 가까운 버스 정류장 X좌표', '가장 가까운 버스 정류장 Y좌표', '동', 
         '가장 가까운 다리 index','가장 가까운 다리', '구', '호선'], axis=1, inplace=True)

In [63]:
df.columns

Index(['전용면적', '계약년월', '계약일', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test',
       '가장 가까운 거리', '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년',
       '계약월', 'GDP', '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량',
       '건설사 랭킹', '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수',
       '거래활발지수', '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리',
       '구 카테고리', '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수',
       '인근 다리 개수 1개이상 3개 이하', '학군'],
      dtype='object')

In [64]:
df.shape

(1128094, 40)

In [65]:
print(df.shape)
df.head(10)

(1128094, 40)


,전용면적,계약년월,계약일,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,...,매매가격 지수 증감률,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군
0,79.97,201712,8,3,1987,127.05721,37.476763,124000.0,0,1127.738351,...,1.243615,70.0590,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
1,79.97,201712,22,4,1987,127.05721,37.476763,123500.0,0,1127.738351,...,1.243615,70.0590,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
2,54.98,201712,28,5,1987,127.05721,37.476763,91500.0,0,1127.738351,...,1.243615,70.0590,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
3,79.97,201801,3,4,1987,127.05721,37.476763,130000.0,0,1127.738351,...,2.459291,69.3220,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
4,79.97,201801,8,2,1987,127.05721,37.476763,117000.0,0,1127.738351,...,2.459291,69.3220,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
5,79.97,201801,11,1,1987,127.05721,37.476763,130000.0,0,1127.738351,...,2.459291,69.3220,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
6,79.97,201803,19,2,1987,127.05721,37.476763,139500.0,0,1127.738351,...,2.180755,67.1875,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
7,54.98,201804,5,5,1987,127.05721,37.476763,107500.0,0,1127.738351,...,1.280568,66.2450,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
8,79.97,201806,28,3,1987,127.05721,37.476763,145000.0,0,1127.738351,...,0.274474,65.3984,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
9,54.98,201807,9,3,1987,127.05721,37.476763,112000.0,0,1127.738351,...,0.218161,64.8121,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0


In [66]:
# 계약월 변수 생성

df['계약월'] = df['계약년월'].astype(str).str[4:].astype(int)
df

,전용면적,계약년월,계약일,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,...,매매가격 지수 증감률,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군
0,79.97,201712,8,3,1987,127.05721,37.476763,124000.0,0,1127.738351,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
1,79.97,201712,22,4,1987,127.05721,37.476763,123500.0,0,1127.738351,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
2,54.98,201712,28,5,1987,127.05721,37.476763,91500.0,0,1127.738351,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
3,79.97,201801,3,4,1987,127.05721,37.476763,130000.0,0,1127.738351,...,2.459291,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
4,79.97,201801,8,2,1987,127.05721,37.476763,117000.0,0,1127.738351,...,2.459291,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,84.65,202307,19,13,2014,127.10672,37.618870,NaN,1,732.482898,...,-0.641286,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128090,84.62,202307,25,12,2014,127.10672,37.618870,NaN,1,732.482898,...,-0.641286,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128091,101.65,202308,27,12,2014,127.10672,37.618870,NaN,1,732.482898,...,-0.607676,50.991003,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128092,84.94,202309,2,18,2014,127.10672,37.618870,NaN,1,732.482898,...,-0.225827,51.082553,기타,기타,기타,기타,5918.624352,0.0,0,1.0


In [67]:
# 계약년월, 계약일, 건축년도, 계약년 변수 제거

df.drop(['계약년월', '계약일'], axis=1, inplace=True)
df

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,...,매매가격 지수 증감률,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군
0,79.97,3,1987,127.05721,37.476763,124000.0,0,1127.738351,2.0,61.89584,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
1,79.97,4,1987,127.05721,37.476763,123500.0,0,1127.738351,2.0,61.89584,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
2,54.98,5,1987,127.05721,37.476763,91500.0,0,1127.738351,2.0,61.89584,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
3,79.97,4,1987,127.05721,37.476763,130000.0,0,1127.738351,2.0,61.89584,...,2.459291,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
4,79.97,2,1987,127.05721,37.476763,117000.0,0,1127.738351,2.0,61.89584,...,2.459291,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,84.65,13,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,-0.641286,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128090,84.62,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,-0.641286,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128091,101.65,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,-0.607676,50.991003,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128092,84.94,18,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,-0.225827,51.082553,기타,기타,기타,기타,5918.624352,0.0,0,1.0


In [68]:
df['계약월_sin'] = np.sin(2 * np.pi * df['계약월'] / 12)

In [69]:
df.drop(['계약월'], axis=1, inplace=True)
df

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,...,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군,계약월_sin
0,79.97,3,1987,127.05721,37.476763,124000.0,0,1127.738351,2.0,61.89584,...,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0,-2.449294e-16
1,79.97,4,1987,127.05721,37.476763,123500.0,0,1127.738351,2.0,61.89584,...,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0,-2.449294e-16
2,54.98,5,1987,127.05721,37.476763,91500.0,0,1127.738351,2.0,61.89584,...,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0,-2.449294e-16
3,79.97,4,1987,127.05721,37.476763,130000.0,0,1127.738351,2.0,61.89584,...,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0,5.000000e-01
4,79.97,2,1987,127.05721,37.476763,117000.0,0,1127.738351,2.0,61.89584,...,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0,5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,84.65,13,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0,-5.000000e-01
1128090,84.62,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0,-5.000000e-01
1128091,101.65,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,50.991003,기타,기타,기타,기타,5918.624352,0.0,0,1.0,-8.660254e-01
1128092,84.94,18,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,51.082553,기타,기타,기타,기타,5918.624352,0.0,0,1.0,-1.000000e+00


In [70]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군',
       '계약월_sin'],
      dtype='object')

In [71]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군',
       '계약월_sin'],
      dtype='object')

# scaling

In [72]:
def scailing(col, scaler_type):
    if scaler_type == 'min_max': scaler = MinMaxScaler()
    else : scaler = StandardScaler()

    return scaler.fit_transform(df[[col]])

cols = ['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율',
       '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하',
       '계약월_sin']

for col in cols:
    print(col)
    scaler_type = 'min_max'
    if col == '계약월_sin': scaler_type = 'standard'

    df[[col]] = scailing(col, scaler_type)

전용면적
층
건축년도
좌표X
좌표Y
가장 가까운 거리
인근 지하철 역 개수
가장 가까운 버스 정류장 거리
인근 버스 정류장 개수
계약년
GDP
한국은행 기준금리
기대 인플레이션
지가지수
아파트 인허가
미분양
거래량
구별 지가지수
공시지가 평균
매수우위지수
건물나이
구매력지수
거래활발지수
매매가격 지수 증감률
매매 대비 전세가격 비율
가장 가까운 다리와의 거리
인근 한강다리 개수
인근 다리 개수 1개이상 3개 이하
계약월_sin


In [73]:
df

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,...,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군,계약월_sin
0,0.168839,0.095890,0.419355,0.678243,0.167126,124000.0,0,0.353721,0.086957,0.101599,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
1,0.168839,0.109589,0.419355,0.678243,0.167126,123500.0,0,0.353721,0.086957,0.101599,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
2,0.108520,0.123288,0.419355,0.678243,0.167126,91500.0,0,0.353721,0.086957,0.101599,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
3,0.168839,0.109589,0.419355,0.678243,0.167126,130000.0,0,0.353721,0.086957,0.101599,...,0.844200,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,0.684200
4,0.168839,0.082192,0.419355,0.678243,0.167126,117000.0,0,0.353721,0.086957,0.101599,...,0.844200,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,0.684200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,0.180135,0.232877,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,0.345867,기타,기타,기타,기타,0.370157,0.0,0.0,1.0,-0.728745
1128090,0.180063,0.219178,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,0.345867,기타,기타,기타,기타,0.370157,0.0,0.0,1.0,-0.728745
1128091,0.221168,0.219178,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,0.347283,기타,기타,기타,기타,0.370157,0.0,0.0,1.0,-1.245919
1128092,0.180835,0.301370,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,0.349765,기타,기타,기타,기타,0.370157,0.0,0.0,1.0,-1.435218


# 호선 one-hot encoding

In [281]:
ohe = OneHotEncoder(sparse=False)
ohe_re = ohe.fit_transform(df[['호선']])

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
df = pd.concat([df, pd.DataFrame(ohe_re, columns=[col for col in ohe.categories_[0]])], axis=1)

In [22]:
df

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,호선,인근 지하철 역 개수,...,경춘선,공항철도1호선,과천선,분당선,신림선,신분당선,신분당선(연장2),우이신설선,일산선,중앙선
0,0.168839,0.095890,0.419355,0.678243,0.167126,124000.0,0,0.353721,분당선,0.086957,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.168839,0.109589,0.419355,0.678243,0.167126,123500.0,0,0.353721,분당선,0.086957,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.108520,0.123288,0.419355,0.678243,0.167126,91500.0,0,0.353721,분당선,0.086957,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.168839,0.109589,0.419355,0.678243,0.167126,130000.0,0,0.353721,분당선,0.086957,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.168839,0.082192,0.419355,0.678243,0.167126,117000.0,0,0.353721,분당선,0.086957,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,0.180135,0.232877,0.854839,0.807949,0.728162,NaN,1,0.227756,경춘선,0.173913,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128090,0.180063,0.219178,0.854839,0.807949,0.728162,NaN,1,0.227756,경춘선,0.173913,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128091,0.221168,0.219178,0.854839,0.807949,0.728162,NaN,1,0.227756,경춘선,0.173913,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128092,0.180835,0.301370,0.854839,0.807949,0.728162,NaN,1,0.227756,경춘선,0.173913,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df.drop(['호선'], axis=1, inplace=True)
df

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,...,경춘선,공항철도1호선,과천선,분당선,신림선,신분당선,신분당선(연장2),우이신설선,일산선,중앙선
0,0.168839,0.095890,0.419355,0.678243,0.167126,124000.0,0,0.353721,0.086957,0.101599,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.168839,0.109589,0.419355,0.678243,0.167126,123500.0,0,0.353721,0.086957,0.101599,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.108520,0.123288,0.419355,0.678243,0.167126,91500.0,0,0.353721,0.086957,0.101599,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.168839,0.109589,0.419355,0.678243,0.167126,130000.0,0,0.353721,0.086957,0.101599,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.168839,0.082192,0.419355,0.678243,0.167126,117000.0,0,0.353721,0.086957,0.101599,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,0.180135,0.232877,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128090,0.180063,0.219178,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128091,0.221168,0.219178,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128092,0.180835,0.301370,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '거래량', '구', '구별 지가지수', '공시지가 평균', '매수우위지수',
       '30년이상50년이하', '아파트 카테고리', '지하철 카테고리', '구 카테고리', '계약월_sin', '1호선', '2호선',
       '3호선', '4호선', '5호선', '6호선', '7호선', '8호선', '9호선', '9호선(연장)', '경부선',
       '경원선', '경의중앙선', '경인선', '경춘선', '공항철도1호선', '과천선', '분당선', '신림선', '신분당선',
       '신분당선(연장2)', '우이신설선', '일산선', '중앙선'],
      dtype='object')

In [25]:
df.drop(['9호선(연장)', '경부선', '경의중앙선', '신분당선',
         '신림선', '신분당선(연장2)', '중앙선', '8호선',
         '우이신설선', '공항철도1호선', '경인선',
         '1호선','일산선', '과천선', '경춘선'], axis=1, inplace=True)

In [61]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '거래량', '구', '구별 지가지수', '공시지가 평균', '매수우위지수',
       '30년이상50년이하', '계약월_sin', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선',
       '9호선', '경원선', '분당선'],
      dtype='object')

# 카테고리 변수 처리
- 30년이상50년이하
- 아파트 카테고리
- 지하철 카테고리
- 구 카테고리
- 건설사 카테고리
- 인근 다리 개수 1개이상 3개 이하
- 학군

In [74]:
df.head()

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,...,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군,계약월_sin
0,0.168839,0.095890,0.419355,0.678243,0.167126,124000.0,0,0.353721,0.086957,0.101599,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
1,0.168839,0.109589,0.419355,0.678243,0.167126,123500.0,0,0.353721,0.086957,0.101599,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
2,0.108520,0.123288,0.419355,0.678243,0.167126,91500.0,0,0.353721,0.086957,0.101599,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
3,0.168839,0.109589,0.419355,0.678243,0.167126,130000.0,0,0.353721,0.086957,0.101599,...,0.844200,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,0.684200
4,0.168839,0.082192,0.419355,0.678243,0.167126,117000.0,0,0.353721,0.086957,0.101599,...,0.844200,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,0.684200


In [75]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군',
       '계약월_sin'],
      dtype='object')

In [76]:
df['30년이상50년이하'] = df['30년이상50년이하'].astype('category')
df['아파트 카테고리'] = df['아파트 카테고리'].astype('category')
df['지하철 카테고리'] = df['지하철 카테고리'].astype('category')
df['구 카테고리'] = df['구 카테고리'].astype('category')
df['건설사 카테고리'] = df['건설사 카테고리'].astype('category')
df['인근 다리 개수 1개이상 3개 이하'] = df['인근 다리 개수 1개이상 3개 이하'].astype('category')
df['학군'] = df['학군'].astype('category')

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128094 entries, 0 to 1128093
Data columns (total 38 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   전용면적                 1128094 non-null  float64 
 1   층                    1128094 non-null  float64 
 2   건축년도                 1128094 non-null  float64 
 3   좌표X                  1128094 non-null  float64 
 4   좌표Y                  1128094 non-null  float64 
 5   target               1118822 non-null  float64 
 6   is_test              1128094 non-null  int64   
 7   가장 가까운 거리            1128094 non-null  float64 
 8   인근 지하철 역 개수          1128094 non-null  float64 
 9   가장 가까운 버스 정류장 거리     1128094 non-null  float64 
 10  인근 버스 정류장 개수         1128094 non-null  float64 
 11  계약년                  1128094 non-null  float64 
 12  GDP                  1128094 non-null  float64 
 13  한국은행 기준금리            1128094 non-null  float64 
 14  기대 인플레이션             1128094 non-n

# Dataset 생성

## train, test 분할

In [92]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군',
       '계약월_sin'],
      dtype='object')

In [93]:
# 추가한 feature들 중 일부 제거
df_dropped = df.drop([], axis=1)

In [94]:
train_df = df_dropped[df_dropped['is_test'] == 0]
test_df = df_dropped[df_dropped['is_test'] == 1]

In [95]:
print(train_df.shape)
print(test_df.shape)

(1118822, 38)
(9272, 38)


In [96]:
train_df.drop(['is_test'], axis=1, inplace=True)
test_df.drop(['is_test'], axis=1, inplace=True)

/tmp/ipykernel_472052/1571330888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(['is_test'], axis=1, inplace=True)
/tmp/ipykernel_472052/1571330888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['is_test'], axis=1, inplace=True)


In [97]:
print(train_df.shape)
print(test_df.shape)

(1118822, 37)
(9272, 37)


In [98]:
train_X = train_df.drop(['target'], axis=1)
train_y = train_df[['target']]

In [99]:
train_X

,전용면적,층,건축년도,좌표X,좌표Y,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,인근 버스 정류장 개수,계약년,...,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군,계약월_sin
0,0.168839,0.095890,0.419355,0.678243,0.167126,0.353721,0.086957,0.101599,0.200000,0.6250,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
1,0.168839,0.109589,0.419355,0.678243,0.167126,0.353721,0.086957,0.101599,0.200000,0.6250,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
2,0.108520,0.123288,0.419355,0.678243,0.167126,0.353721,0.086957,0.101599,0.200000,0.6250,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
3,0.168839,0.109589,0.419355,0.678243,0.167126,0.353721,0.086957,0.101599,0.200000,0.6875,...,0.844200,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,0.684200
4,0.168839,0.082192,0.419355,0.678243,0.167126,0.353721,0.086957,0.101599,0.200000,0.6875,...,0.844200,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,0.684200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118817,0.120492,0.205479,0.596774,0.281155,0.704836,0.326389,0.086957,0.131622,0.228571,0.0000,...,0.131260,기타,6호선,기타,현대,0.383051,0.0,0.0,2.0,-0.728745
1118818,0.120492,0.191781,0.596774,0.281155,0.704836,0.326389,0.086957,0.131622,0.228571,0.0000,...,0.127898,기타,6호선,기타,현대,0.383051,0.0,0.0,2.0,-1.245919
1118819,0.180570,0.328767,0.596774,0.281155,0.704836,0.326389,0.086957,0.131622,0.228571,0.0000,...,0.127898,기타,6호선,기타,현대,0.383051,0.0,0.0,2.0,-1.245919
1118820,0.180570,0.164384,0.596774,0.281155,0.704836,0.326389,0.086957,0.131622,0.228571,0.0000,...,0.125540,기타,6호선,기타,현대,0.383051,0.0,0.0,2.0,-1.435218


In [100]:
train_X.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리', '인근 지하철 역 개수',
       '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP', '한국은행 기준금리',
       '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹', '구별 지가지수',
       '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군',
       '계약월_sin'],
      dtype='object')

In [101]:
train_y

,target
0,124000.0
1,123500.0
2,91500.0
3,130000.0
4,117000.0
...,...
1118817,20000.0
1118818,20000.0
1118819,28000.0
1118820,29000.0


## train, valid 분할

In [102]:
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, train_size=0.95, test_size=0.05, random_state=74)

print(train_X.shape, train_y.shape)
print(valid_X.shape, valid_y.shape)

(1062880, 36) (1062880, 1)
(55942, 36) (55942, 1)


In [103]:
train_X.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리', '인근 지하철 역 개수',
       '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP', '한국은행 기준금리',
       '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹', '구별 지가지수',
       '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군',
       '계약월_sin'],
      dtype='object')

# model

In [104]:
params = {
    'n_estimators': 100000,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'metric_freq': 20,
    'device': 'gpu',
    'verbosity': 0
}

model = lgb.LGBMRegressor(n_estimators=100000,
                          metric='rmse', data_sample_strategy='goss'
                          )

model.fit(
    train_X, train_y,
    eval_set = [(train_X, train_y), (valid_X, valid_y)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=60),
               lgb.log_evaluation(period=10, show_stdv=True)]
)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037846 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4034
[LightGBM] [Info] Number of data points in the train set: 1062880, number of used features: 36
[LightGBM] [Info] Using GOSS
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 58121.472893
Training until validation scores don't improve for 60 rounds
[10]	training's rmse: 27451.4	valid_1's rmse: 27624.7
[20]	training's rmse: 21237.2	valid_1's rmse: 21199.2
[30]	training's rmse: 18686.2	valid_1's rmse: 18626.4
[40]	training's rmse: 17330.6	valid_1's rmse: 17297.5
[50]	training's rmse: 16542.5	valid_1's rmse: 16538.7
[60]	training's rmse: 15952.7	valid_1's rmse: 15995.4
[70]	training'

LGBMRegressor(data_sample_strategy='goss', metric='rmse', n_estimators=100000)

In [86]:
joblib.dump(model, '../models/feature_test.pkl')

['../models/feature_test.pkl']

In [87]:
train_X.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리', '인근 지하철 역 개수',
       '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP', '한국은행 기준금리',
       '기대 인플레이션', '아파트 인허가', '미분양', '거래량', '구별 지가지수', '공시지가 평균', '매수우위지수',
       '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수', '매매가격 지수 증감률', '매매 대비 전세가격 비율',
       '아파트 카테고리', '지하철 카테고리', '구 카테고리', '건설사 카테고리', '가장 가까운 다리와의 거리',
       '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군', '계약월_sin'],
      dtype='object')

# submission

In [40]:
test_df.drop(['target'], axis=1, inplace=True)

/tmp/ipykernel_456466/1820151961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['target'], axis=1, inplace=True)


In [41]:
test_df.drop(['구'], axis=1, inplace=True)

/tmp/ipykernel_456466/2563928711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['구'], axis=1, inplace=True)


### 예측

In [42]:
preds = model.predict(test_df)

In [43]:
preds

array([182324.3413283 , 266889.63275921, 303469.33644405, ...,
        84957.28600137,  76530.75532449,  77456.33285716])

# 결과값 처리
- float 형태에서 int 형태로 변환 (반올림 사용)
- 결과 순서를 맞춰줘야 된다. -> 기존 test 데이터에 target column을 추가하고 거기에 결과를 저장한다.
- 모든 결과를 concat하고 인덱스를 기준으로 정렬한다.

In [44]:
preds = np.round(preds).astype(int)

In [45]:
preds_df = pd.DataFrame(preds, columns=['target'])

In [46]:
preds_df

,target
0,182324
1,266890
2,303469
3,264446
4,221255
...,...
9267,75345
9268,74363
9269,84957
9270,76531


In [47]:
preds_df

,target
0,182324
1,266890
2,303469
3,264446
4,221255
...,...
9267,75345
9268,74363
9269,84957
9270,76531


In [48]:
preds_df.to_csv('../preds/v_27_preds.csv', index=False)